In [1]:
import os
import tensorflow as tf 
import io
import pandas as pd
from object_detection.utils import dataset_util
from PIL import Image
from time import time

In [2]:
#cwd = os.getcwd()
# new a tensorflow model
sess = tf.Session()
# make a placeholder more flexible
encoded_jpg_ph = tf.placeholder(tf.string, shape=[])
# set resize layer
height = 256
width = 256	
# resizing the image here
decoded_image = tf.image.decode_jpeg(encoded_jpg_ph)
decoded_image_resized = tf.image.resize_images(decoded_image, [height, width]) # this returns float32
decoded_image_resized_uint = tf.cast(decoded_image_resized, tf.uint8)
resize_image   = tf.image.encode_jpeg(decoded_image_resized_uint) # expects uint8
# reset all variables
sess.run(tf.global_variables_initializer())

In [3]:
def create_cat_tf_example(label, label_text, img_path, img_name, row = None):
    """Creates a tf.Example proto from sample cat image.

    Args:
    encoded_cat_image_data: The jpg encoded data of the cat image.

    Returns:
    example: The created tf.Example.
    """

    with tf.gfile.FastGFile(os.path.join(img_path, img_name), 'rb') as fid:
        encoded_image = fid.read() 

    encoded_image_data = sess.run(resize_image, {encoded_jpg_ph: encoded_image}) #  I think this may not be the right way of doing this
    b_filename = str.encode(img_name)

    image_format = b'jpg'
    if not row is None:
        xmins = [row["xmin"]]
        xmaxs = [row["xmax"]]
        ymins = [row["ymin"]]
        ymaxs = [row["ymax"]]
    else:
        xmins = [10.0 / width]
        xmaxs = [(width - 10) / width]
        ymins = [10.0 / height]
        ymaxs = [(height - 10.0) / height]    
    # classes_text = [str.encode(label_text)]
    classes_text = []
    if label_text:
        classes_text.append(label_text.encode('utf8'))
    classes = []
    # if label == 1:
    classes.append(int(label))
    # print(classes_text, classes, b_filename)
    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(b_filename),
        'image/source_id': dataset_util.bytes_feature(b_filename),
        'image/encoded': dataset_util.bytes_feature(encoded_image_data),
        # 'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example

In [4]:
start_time = time()
each_batch_time = time()
box_df = pd.read_csv(os.path.join(".", "auto_box", "auto_box.csv"))


In [5]:
box_df.head()

,filename,width,height,class,xmin,ymin,xmax,ymax,score
0,dog.0.jpg,499,375,dog,0.157638,0.093376,0.739552,0.805292,97.889382
1,dog.1.jpg,327,499,dog,0.095817,0.096760,0.886062,0.960047,96.295798
2,dog.10.jpg,269,292,dog,0.059631,0.069795,0.923171,0.989220,87.839001
3,dog.100.jpg,500,397,dog,0.071499,0.073221,0.735138,0.987441,95.074093
4,dog.1000.jpg,499,375,dog,0.231863,0.216221,0.583986,0.651290,99.176532


In [7]:
# collect the dirs
mode_list = ["train", "eval"]
for mode in mode_list:
    cwd = os.path.join("./dataset/self_divide", mode)
    writer = tf.python_io.TFRecordWriter("./dataset/TFRECORD/" + mode + ".tfrecords")
    for index, row in box_df.iterrows():
        class_path = os.path.join(cwd, row["class"])
        img_name = row["filename"]
        name = row["class"]
        if (index % 100 == 0):
            output_str = mode + " step -- " + str(index)
            print(output_str, " compute 100 image_batch time = ", time() - each_batch_time)
            print("id =", int(index+1),name, row["filename"], class_path)
            each_batch_time = time()
        if(os.path.exists(os.path.join(class_path, img_name))):
            if(name == "dog"):
                each_record = create_cat_tf_example(label = 1, label_text = name, img_path = class_path, img_name = img_name, row = row)
            elif (name == "cat"):
                each_record = create_cat_tf_example(label = 2, label_text = name, img_path = class_path, img_name = img_name, row = row)

            writer.write(each_record.SerializeToString())  #序列化为字符串
    writer.close()
    print(mode , "is finished.")
sess.close()
print("cost time =", time() - start_time)

train step -- 0  compute 100 image_batch time =  19.626466751098633
id = 1 dog dog.0.jpg ./dataset/self_divide\train\dog
train step -- 100  compute 100 image_batch time =  0.7409539222717285
id = 101 dog dog.10117.jpg ./dataset/self_divide\train\dog
train step -- 200  compute 100 image_batch time =  0.7549359798431396
id = 201 dog dog.10228.jpg ./dataset/self_divide\train\dog
train step -- 300  compute 100 image_batch time =  0.7156667709350586
id = 301 dog dog.10346.jpg ./dataset/self_divide\train\dog
train step -- 400  compute 100 image_batch time =  0.7497823238372803
id = 401 dog dog.10462.jpg ./dataset/self_divide\train\dog
train step -- 500  compute 100 image_batch time =  0.6978855133056641
id = 501 dog dog.10582.jpg ./dataset/self_divide\train\dog
train step -- 600  compute 100 image_batch time =  0.828338623046875
id = 601 dog dog.10688.jpg ./dataset/self_divide\train\dog
train step -- 700  compute 100 image_batch time =  0.8107247352600098
id = 701 dog dog.10799.jpg ./dataset

train step -- 6400  compute 100 image_batch time =  0.7281250953674316
id = 6401 dog dog.5952.jpg ./dataset/self_divide\train\dog
train step -- 6500  compute 100 image_batch time =  0.750495433807373
id = 6501 dog dog.6059.jpg ./dataset/self_divide\train\dog
train step -- 6600  compute 100 image_batch time =  0.7515380382537842
id = 6601 dog dog.6175.jpg ./dataset/self_divide\train\dog
train step -- 6700  compute 100 image_batch time =  0.705172061920166
id = 6701 dog dog.629.jpg ./dataset/self_divide\train\dog
train step -- 6800  compute 100 image_batch time =  0.7003672122955322
id = 6801 dog dog.6406.jpg ./dataset/self_divide\train\dog
train step -- 6900  compute 100 image_batch time =  0.6920311450958252
id = 6901 dog dog.6509.jpg ./dataset/self_divide\train\dog
train step -- 7000  compute 100 image_batch time =  0.6727621555328369
id = 7001 dog dog.6618.jpg ./dataset/self_divide\train\dog
train step -- 7100  compute 100 image_batch time =  0.6749680042266846
id = 7101 dog dog.6726

train step -- 12700  compute 100 image_batch time =  0.7333767414093018
id = 12701 cat cat.1806.jpg ./dataset/self_divide\train\cat
train step -- 12800  compute 100 image_batch time =  0.7320761680603027
id = 12801 cat cat.1925.jpg ./dataset/self_divide\train\cat
train step -- 12900  compute 100 image_batch time =  0.7331397533416748
id = 12901 cat cat.2038.jpg ./dataset/self_divide\train\cat
train step -- 13000  compute 100 image_batch time =  0.7180576324462891
id = 13001 cat cat.2154.jpg ./dataset/self_divide\train\cat
train step -- 13100  compute 100 image_batch time =  0.8260784149169922
id = 13101 cat cat.2269.jpg ./dataset/self_divide\train\cat
train step -- 13200  compute 100 image_batch time =  0.7708878517150879
id = 13201 cat cat.2384.jpg ./dataset/self_divide\train\cat
train step -- 13300  compute 100 image_batch time =  0.735278844833374
id = 13301 cat cat.2498.jpg ./dataset/self_divide\train\cat
train step -- 13400  compute 100 image_batch time =  0.7618868350982666
id = 

train step -- 19000  compute 100 image_batch time =  0.7334604263305664
id = 19001 cat cat.8868.jpg ./dataset/self_divide\train\cat
train step -- 19100  compute 100 image_batch time =  0.7790687084197998
id = 19101 cat cat.8972.jpg ./dataset/self_divide\train\cat
train step -- 19200  compute 100 image_batch time =  0.737145185470581
id = 19201 cat cat.9081.jpg ./dataset/self_divide\train\cat
train step -- 19300  compute 100 image_batch time =  0.7196214199066162
id = 19301 cat cat.9193.jpg ./dataset/self_divide\train\cat
train step -- 19400  compute 100 image_batch time =  0.7611844539642334
id = 19401 cat cat.9311.jpg ./dataset/self_divide\train\cat
train step -- 19500  compute 100 image_batch time =  0.7196736335754395
id = 19501 cat cat.9425.jpg ./dataset/self_divide\train\cat
train step -- 19600  compute 100 image_batch time =  0.7480957508087158
id = 19601 cat cat.955.jpg ./dataset/self_divide\train\cat
train step -- 19700  compute 100 image_batch time =  0.7186641693115234
id = 1

eval step -- 500  compute 100 image_batch time =  0.021484851837158203
id = 501 dog dog.10582.jpg ./dataset/self_divide\eval\dog
eval step -- 600  compute 100 image_batch time =  0.012037515640258789
id = 601 dog dog.10688.jpg ./dataset/self_divide\eval\dog
eval step -- 700  compute 100 image_batch time =  0.01102590560913086
id = 701 dog dog.10799.jpg ./dataset/self_divide\eval\dog
eval step -- 800  compute 100 image_batch time =  0.011031866073608398
id = 801 dog dog.10913.jpg ./dataset/self_divide\eval\dog
eval step -- 900  compute 100 image_batch time =  0.005248069763183594
id = 901 dog dog.11024.jpg ./dataset/self_divide\eval\dog
eval step -- 1000  compute 100 image_batch time =  0.015657901763916016
id = 1001 dog dog.11135.jpg ./dataset/self_divide\eval\dog
eval step -- 1100  compute 100 image_batch time =  0.0
id = 1101 dog dog.11244.jpg ./dataset/self_divide\eval\dog
eval step -- 1200  compute 100 image_batch time =  0.015596866607666016
id = 1201 dog dog.1136.jpg ./dataset/se

eval step -- 8100  compute 100 image_batch time =  0.02068924903869629
id = 8101 dog dog.7862.jpg ./dataset/self_divide\eval\dog
eval step -- 8200  compute 100 image_batch time =  0.011028289794921875
id = 8201 dog dog.7973.jpg ./dataset/self_divide\eval\dog
eval step -- 8300  compute 100 image_batch time =  0.011055707931518555
id = 8301 dog dog.8083.jpg ./dataset/self_divide\eval\dog
eval step -- 8400  compute 100 image_batch time =  0.0110321044921875
id = 8401 dog dog.8192.jpg ./dataset/self_divide\eval\dog
eval step -- 8500  compute 100 image_batch time =  0.011029243469238281
id = 8501 dog dog.8299.jpg ./dataset/self_divide\eval\dog
eval step -- 8600  compute 100 image_batch time =  0.0020923614501953125
id = 8601 dog dog.841.jpg ./dataset/self_divide\eval\dog
eval step -- 8700  compute 100 image_batch time =  0.015655517578125
id = 8701 dog dog.8520.jpg ./dataset/self_divide\eval\dog
eval step -- 8800  compute 100 image_batch time =  0.01465606689453125
id = 8801 dog dog.8634.jp

eval step -- 15200  compute 100 image_batch time =  0.011030435562133789
id = 15201 cat cat.4614.jpg ./dataset/self_divide\eval\cat
eval step -- 15300  compute 100 image_batch time =  0.012005805969238281
id = 15301 cat cat.4722.jpg ./dataset/self_divide\eval\cat
eval step -- 15400  compute 100 image_batch time =  0.012032270431518555
id = 15401 cat cat.4831.jpg ./dataset/self_divide\eval\cat
eval step -- 15500  compute 100 image_batch time =  0.012031793594360352
id = 15501 cat cat.4946.jpg ./dataset/self_divide\eval\cat
eval step -- 15600  compute 100 image_batch time =  0.011029481887817383
id = 15601 cat cat.5058.jpg ./dataset/self_divide\eval\cat
eval step -- 15700  compute 100 image_batch time =  0.010018110275268555
id = 15701 cat cat.5164.jpg ./dataset/self_divide\eval\cat
eval step -- 15800  compute 100 image_batch time =  0.006145477294921875
id = 15801 cat cat.5278.jpg ./dataset/self_divide\eval\cat
eval step -- 15900  compute 100 image_batch time =  0.015653610229492188
id 

eval step -- 21600  compute 100 image_batch time =  0.9308078289031982
id = 21601 dog dog.5896.jpg ./dataset/self_divide\eval\dog
eval step -- 21700  compute 100 image_batch time =  0.9008910655975342
id = 21701 dog dog.6369.jpg ./dataset/self_divide\eval\dog
eval step -- 21800  compute 100 image_batch time =  0.9467253684997559
id = 21801 dog dog.6897.jpg ./dataset/self_divide\eval\dog
eval step -- 21900  compute 100 image_batch time =  0.9503037929534912
id = 21901 dog dog.7351.jpg ./dataset/self_divide\eval\dog
eval step -- 22000  compute 100 image_batch time =  0.9499139785766602
id = 22001 dog dog.7764.jpg ./dataset/self_divide\eval\dog
eval step -- 22100  compute 100 image_batch time =  0.9030160903930664
id = 22101 dog dog.8249.jpg ./dataset/self_divide\eval\dog
eval step -- 22200  compute 100 image_batch time =  0.9156148433685303
id = 22201 dog dog.8700.jpg ./dataset/self_divide\eval\dog
eval step -- 22300  compute 100 image_batch time =  0.9633321762084961
id = 22301 dog dog.